In [1]:
!pip install huggingface



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tortch (from versions: none)
ERROR: No matching distribution found for tortch


In [2]:
!pip install torch


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 5.2 MB/s 
     |████████████████████████████████| 163 kB 35.0 MB/s 
     |████████████████████████████████| 7.6 MB 34.9 MB/s 


In [6]:
from transformers import AutoModel,AutoTokenizer

tok = AutoTokenizer.from_pretrained('KBLab/bert-base-swedish-cased')
model = AutoModel.from_pretrained('KBLab/bert-base-swedish-cased')



Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/491 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at KBLab/bert-base-swedish-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
text = "Skriv en mening här, vad som helst, på svenska."
marked_text = "[CLS] " + text + " [SEP]"

# Tokenizera texten med BERT tokenizer.
tokenized_text = tok.tokenize(marked_text)

# Skriv ut genererade tokens.
print (tokenized_text)

['[CLS]', 'Skriv', 'en', 'mening', 'här', ',', 'vad', 'som', 'helst', ',', 'på', 'engelska', '.', '[SEP]']


In [10]:
#Lista lite ord som finns i base BERT:s vokabulär
list(tok.vocab.keys())[5000:5020]

['##alvatten',
 'Jörg',
 'Tango',
 'repress',
 '374',
 'gun',
 'begått',
 '##gränd',
 'Plötsligt',
 'verka',
 'Trollhätt',
 '##xf',
 'sf',
 '##frågningar',
 'tages',
 '##osamt',
 '##producerade',
 'sprit',
 '##ENTAR',
 'byråd']

In [30]:
# Skriv en ny mening där du inkluderar ett ord som har olika betydelse
text = "Efter att ha stulit pengar på en bank, syntes bankrånaren fiska på Missisippis flodbank."
      

# Lägg till de viktiga specialtokens som BERT kräver.
marked_text = "[CLS] " + text + " [SEP] " 

# Dela upp meningarna i tokens.
tokenized_text = tok.tokenize(marked_text)

# Mappa tokens till index i vokabuläret 
indexed_tokens = tok.convert_tokens_to_ids(tokenized_text)

# Visa orden med deras index 
for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

[CLS]             2
Efter           982
att              48
ha              331
stulit       24,459
pengar        1,702
på               68
en               59
bank          2,419
,                19
syntes       13,443
bankrån      43,883
##aren          322
fiska        22,700
på               68
Miss          7,422
##isi        26,464
##pp             78
##is            116
flod         15,672
##bank        7,613
.                 7
[SEP]             3


In [31]:
# Vi har bara en mening, alltså tillhör alla våra tokens segment_id 1
segments_ids = [1] * len(tokenized_text)

print (segments_ids)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [32]:
# Konvertera input till PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

In [33]:
# Ladda den förtränade modellen (vikterna för modellen)  
model = BertModel.from_pretrained('KBLab/bert-base-swedish-cased',
                                  output_hidden_states = True, # Om modellen ska returnera alla hidden states dvs alla vektorer som skapats
                                  )

# Feed forward mode, kallas för evaluation mode. Det finns flera andra för att utföra andra uppgifter 
model.eval()

Some weights of the model checkpoint at KBLab/bert-base-swedish-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(50325, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [34]:
# Kör din text genom BERT. och samla alla hidden states som producerats från alla 12 lager i BERT. no_grad nedan innebär att vi inte räknar ut hur det gick, 
# eftersom vi inte ska bakåtpropagera, eftersom vi inte tränar, sparar minne.  
with torch.no_grad():

    outputs = model(tokens_tensor, segments_tensors)

    # När vi utvärderar modellen kommer den returnera ett antal objekt, beroende på hur den är konfigurerad. Eftersom vi satt output_hidden_state = True
    # Så kommer det tredje objektet att vara hidden state från alla lager. Se dokumentationen för detlajer: https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]

In [35]:
# Ta bort dimensionen som representerar batches eftersom vi bara har en mening.
token_embeddings = torch.stack(hidden_states, dim=0)

token_embeddings.size()

torch.Size([13, 1, 23, 768])

In [36]:
token_embeddings = torch.squeeze(token_embeddings, dim=1)

token_embeddings.size()

torch.Size([13, 23, 768])

In [37]:
# Byt plats på dimension 0 och 1 för att få tokens först i tensorn (grupperad på tokens)
token_embeddings = token_embeddings.permute(1,0,2)

token_embeddings.size()

torch.Size([23, 13, 768])

In [38]:
# Spara token vektorer, med storlek [22 * 3 072]
token_vecs_cat = []

# `Embeddings för tokens är en tensor med dimensionerna [22 x 12 x 768] 

# För varje token i vår mening...
for token in token_embeddings:
    
    # `token` är en [12 x 768] tensor

    # Conkatenera vektorerna
    cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
    
    # Använd `cat_vec` för att representera `token`.
    token_vecs_cat.append(cat_vec)

print ('Storleken är: %d x %d' % (len(token_vecs_cat), len(token_vecs_cat[0])))

Storleken är: 23 x 3072


In [39]:
# Spara tokenvectorer med storlek [22 x 768]
token_vecs_sum = []

# `token_embeddings` är en [22 x 12 x 768] tensor.


for token in token_embeddings:
    #Summera de fyra sista lagren
    sum_vec = torch.sum(token[-4:], dim=0)
    token_vecs_sum.append(sum_vec)

print ('Storleken är: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))

Storleken är: 23 x 768


In [40]:
# `hidden_states` har storleken [13 x 1 x 22 x 768]

# `token_vecs` är en tensor med storleken [22 x 768]
token_vecs = hidden_states[-2][0]

# Beräkna medelvärdet för alla 22 tokens
sentence_embedding = torch.mean(token_vecs, dim=0)

print ("Our final sentence embedding vector of shape:", sentence_embedding.size())

Our final sentence embedding vector of shape: torch.Size([768])


In [41]:
# Vilka index finns vårt ord med flera betydelser på? 
for i, token_str in enumerate(tokenized_text):
  print (i, token_str)

0 [CLS]
1 Efter
2 att
3 ha
4 stulit
5 pengar
6 på
7 en
8 bank
9 ,
10 syntes
11 bankrån
12 ##aren
13 fiska
14 på
15 Miss
16 ##isi
17 ##pp
18 ##is
19 flod
20 ##bank
21 .
22 [SEP]


In [42]:
print('Första 5 vektorvärdena för varje instans av ordet')
print('')
print("på en bank    ", str(token_vecs_sum[8][:5]))
print("bankrån   ", str(token_vecs_sum[11][:5]))
print("flodbank   ", str(token_vecs_sum[20][:5]))

Första 5 vektorvärdena för varje instans av ordet

på en bank     tensor([-3.5788, -2.9402,  0.5999, -2.3153,  1.8977])
bankrån    tensor([-2.2807, -5.5591,  0.8518,  0.4027,  1.5016])
flodbank    tensor([-2.3475,  0.0335,  2.9463,  1.2498,  1.3469])


In [43]:
from scipy.spatial.distance import cosine

# Beräkna cosinuslikehten mellan ordets olika betydelse 
diff_bank = 1 - cosine(token_vecs_sum[10], token_vecs_sum[19])

# Beräkna för ordet i lika betydelse
same_bank = 1 - cosine(token_vecs_sum[10], token_vecs_sum[6])

print('Likhet för *liknande*  betydelse:  %.2f' % same_bank)
print('Likhet för *olik* betydelse:  %.2f' % diff_bank)

Likhet för *liknande*  betydelse:  0.59
Likhet för *olik* betydelse:  0.68
